In [20]:
import pandas as pd
from geopol_client import GeopolriskConsumer

In [21]:
master_df = pd.read_csv("../data/hhi_production_egsehi_conformed_table.csv")
master_df

,Year,Country,Domestic Production Value (P_AC),Product,ISO3,value_economic,no_value_economic,value_governance,no_value_governance,value_social,no_value_social,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,EGSEHI,EGSEHI_6root
0,1970,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1972,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1974,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69530,2016,RUSSIAN FEDERATION,5.584791e+08,PETROLEUM,RUS,0.624176,False,0.358850,False,0.608937,False,0.414390,False,0.517701,False,0.158387,False,0.004634,9.908522e-15
69531,2017,RUSSIAN FEDERATION,5.585467e+08,PETROLEUM,RUS,0.642995,False,0.365093,False,0.582633,False,0.413218,False,0.513469,False,0.158387,False,0.004596,9.430271e-15
69532,2018,RUSSIAN FEDERATION,5.678849e+08,PETROLEUM,RUS,0.645651,False,0.370905,False,0.607808,False,0.412046,False,0.509131,False,0.158387,False,0.004836,1.279774e-14
69533,2019,RUSSIAN FEDERATION,5.733888e+08,PETROLEUM,RUS,0.670757,False,0.378865,False,0.599214,False,0.413524,False,0.512174,False,0.158387,False,0.005108,1.776735e-14


In [22]:
grc = GeopolriskConsumer()

In [23]:
# Paises de interes
list_parameters = [
    ("Canada", "Lithium", 2010),
    ("Mexico", "Petroleum", 2010),
    ("USA", "Gold", 2011)
]

In [28]:
total_result_df = pd.DataFrame({
            "Year": [], "Country": [], "Domestic Production Value (P_AC)": [], "Product": [],
            "ISO3": [], "value_economic": [], "no_value_economic": [], "value_governance": [],
            "no_value_governance": [], "value_social": [], "no_value_social": [], "value_ecosystems": [],
            "no_value_ecosystems": [], "value_habitat": [], "no_value_habitat": [], "value_infrastructure": [],
            "no_value_infrastructure": [], "EGSEHI": [], "EGSEHI_6root": [], "Reporter": [],"Partner": [], 
            "ISO3 Code": [], "Imports": [], "Total Imports": [], "YearLog": [], "Geopolitical Risk": []
        })

total_condensed_result_df = pd.DataFrame({
            "Product": [], "Country": [], "Year": [], "Geopolitical Risk": []
        })

for tuple in list_parameters:
    country, product, ps = tuple[0], tuple[1], tuple[2]
    # Filter FACT table with user parameters
    valid_rows = (master_df["Country"] == country.upper()) & (master_df["Product"] == product.upper()) & (master_df['Year'] == ps)
    filtered_fact_df = master_df[valid_rows]

    # Get F and f for 
    comtrade_df = grc.query_comtrade_api(country, product, ps)

    # Merge filtered FACT table and comtrade result table
    result_df = pd.merge(left=filtered_fact_df, right=comtrade_df, how="left", left_on=["Year", "Country"], right_on=["YearLog", "Reporter"])
    # result_df["Geopolitical Risk"] = result_df["hhi"] * (result_df["Stability value"] * result_df["Imports"] / (result_df["Domestic Production Value"] + result_df["Total Imports"]))
    result_df["Geopolitical Risk"] = 1 * (result_df["EGSEHI_6root"] * result_df["Imports"] / (result_df["Domestic Production Value (P_AC)"] + result_df["Total Imports"]))

    condensed_result_df = result_df[["Product", "Country", "Year", "Geopolitical Risk"]].groupby(by=["Product", "Country", "Year"], as_index=False).sum()

    total_result_df = pd.concat([total_result_df, result_df])
    total_condensed_result_df = pd.concat([total_condensed_result_df, condensed_result_df])

In [29]:
total_result_df.to_csv("../data/results/results.csv")

In [30]:
total_condensed_result_df

,Product,Country,Year,Geopolitical Risk
0,LITHIUM,CANADA,2010.0,5.332193e-12
0,PETROLEUM,MEXICO,2010.0,9.069571e-22
0,GOLD,USA,2011.0,1.077226e-10
